In [1]:
from benchmark_tools import Dataset, experiment
from darts.models import NaiveSeasonal, NHiTSModel, Prophet
from darts.utils import missing_values


models = [NaiveSeasonal, NHiTSModel, Prophet]

In [2]:
from darts.datasets import AirPassengersDataset, WeatherDataset

air_pass_series = missing_values.fill_missing_values(AirPassengersDataset().load())
weather_ds = missing_values.fill_missing_values(WeatherDataset().load().resample("1h"))[
    -7000:
]
weather_past_cov = weather_ds[
    ["p (mbar)", "wv (m/s)", "wd (deg)", "rain (mm)", "raining (s)", "SWDR (W/m²)"]
]
weather_series = weather_ds["T (degC)"]

list_datasets = [
    Dataset(name="Air passengers", series=air_pass_series),
    Dataset(name="Weather", series=weather_series, past_covariates=weather_past_cov),
]

In [3]:
results = experiment(
    list_datasets=list_datasets,
    models=models,
    grid_search=False,
    forecast_horizon=0.05,
    repeat=5,
    silent_search=True,
)

Using forecast horizon of length 1
#####################################################
Air passengers NaiveSeasonal 45.1
Using forecast horizon of length 1


09:38:02 - cmdstanpy - INFO - Chain [1] start processing
09:38:02 - cmdstanpy - INFO - Chain [1] done processing
09:38:02 - cmdstanpy - INFO - Chain [1] start processing
09:38:02 - cmdstanpy - INFO - Chain [1] done processing
09:38:02 - cmdstanpy - INFO - Chain [1] start processing


#####################################################
Air passengers NHiTSModel 48.97123210509198
Using forecast horizon of length 1


09:38:02 - cmdstanpy - INFO - Chain [1] done processing
09:38:02 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] start processing
09:38:03 - cmdstanpy - INFO - Chain [1] done processing
09:38:03 - cmdstanpy - INFO - Chain [1] 

#####################################################
Air passengers Prophet 33.473200148176694
Using forecast horizon of length 69
#####################################################
Weather NaiveSeasonal 3.4545000000000003
Using forecast horizon of length 69
